In [ ]:
import kfp
from kfp.components import load_component_from_url, create_component_from_func
from kfp.components import InputPath, OutputPath

import sys
sys.path.insert(0, "..")
from constants import NAMESPACE, HOST
from utils.auth import get_session_cookie

In [ ]:
# Ref: https://www.kubeflow.org/docs/components/pipelines/sdk/output-viewer/#web-app
def produce_html(mlpipeline_ui_metadata_path: kfp.components.OutputPath()):
    import json
    import os

    metadata = {
        'outputs' : [{
          'type': 'web-app',
          'storage': 'inline',
          'source': '<h1>Hello, World!</h1>',
        }]
    }

    with open(mlpipeline_ui_metadata_path, 'w') as metadata_file:
        json.dump(metadata, metadata_file)
        
produce_html_op = create_component_from_func(
    produce_html,
    base_image='python:3.7',
    packages_to_install=[],
    output_component_file='../components/visualize_html/component.yaml',
)

In [ ]:
# Define a pipeline and create a task from a component:
def my_pipeline(url):
    produce_html_op()

In [ ]:
session_cookie = get_session_cookie()
client = kfp.Client(
    host=f"{HOST}/pipeline",
    cookies=f"authservice_session={session_cookie}",
    namespace=NAMESPACE,
)
client.create_run_from_pipeline_func(
    my_pipeline,
    arguments={}
)